In [ ]:
%%writefile exemplo.cu

#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/generate.h>
#include <thrust/sort.h>
#include <thrust/copy.h>
#include <thrust/random.h>

int main() {
  // Generate 32M random numbers serially.
  thrust::default_random_engine rng(1337);
  thrust::uniform_int_distribution<int> dist;
  thrust::host_vector<int> h_vec(32 << 20);
  thrust::generate(h_vec.begin(), h_vec.end(), [&] { return dist(rng); });

  // Transfer data to the device.
  thrust::device_vector<int> d_vec = h_vec;

  // Sort data on the device.
  thrust::sort(d_vec.begin(), d_vec.end());

  // Transfer data back to host.
  thrust::copy(d_vec.begin(), d_vec.end(), h_vec.begin());
}

Writing exemplo.cu


In [ ]:
!nvidia-smi

Tue Apr 25 13:54:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#!nvcc -arch=sm_75 -std=c++14 exemplo.cpp -o exemplo

!nvcc exemplo.cu -o exemplo

# essa linha diz que vamos depender do compilador nvcc. E aí sabemos qual tipo de arquitetura estamos usando. No exemplo é o sm_70.Vamos supor que em casa temos uma Geforce rtx 4090
# No Google é a sm_20

In [ ]:
!./exemplo

In [ ]:
%%writefile aula14.cu

#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/generate.h>
#include <thrust/sort.h>
#include <thrust/copy.h>
#include <thrust/random.h>
#include <vector>
using namespace std;

int main() {
  // Generate 32M random numbers serially.
  thrust::default_random_engine rng(1337);
  thrust::uniform_int_distribution<int> dist;
  thrust::host_vector<int> h_vec(50);
  thrust::generate(h_vec.begin(), h_vec.end(), [&] { return dist(rng); });

  // Transfer data to the device.
  thrust::device_vector<int> d_vec = h_vec;

  // Sort data on the device.
  thrust::sort(d_vec.begin(), d_vec.end());



  thrust::device_vector<int>::iterator begin = d_vec.begin();
  thrust::device_vector<int>::iterator end = d_vec.end();
  int len = end - begin;


  double media = thrust::reduce(d_vec.begin(), d_vec.end()) / len;

//   thrust::device_vector<double> media_vector(len, media);

  thrust::constant_iterator<double> media_vector_begin(media);
  thrust::constant_iterator<double> media_vector_end = media_vector_begin + len;

  thrust::device_vector<double> subtracao_vec(len);
  thrust::device_vector<double> quadrado_vec(len);
  thrust::transform (d_vec.begin(), d_vec.end(), media_vector_begin, subtracao_vec.begin(), thrust::minus<double>());
  thrust::transform (subtracao_vec.begin(), subtracao_vec.end(), subtracao_vec.begin(), quadrado_vec.begin(), thrust::multiplies<double>());

  double soma_quadrados = thrust::reduce(quadrado_vec.begin(), quadrado_vec.end());

  double variance = soma_quadrados / len;

  std::cout << "variancia: " << variance << std::endl;
}


Writing aula14.cu


In [ ]:
!nvcc aula14.cu -o aula14

In [ ]:
!./aula14

variancia: 1.23561e+18


In [1]:
%%writefile aula15.cu

 #include <thrust/host_vector.h>
 #include <thrust/device_vector.h>
 #include <thrust/generate.h>
 #include <thrust/functional.h>
 #include <thrust/copy.h>
 #include <cstdlib>
 #include <algorithm>
 #include <iostream>
 #include <iomanip>

 using namespace  std;

struct saxpy
{
    int a;    
    saxpy(int a_) : a(a_) {};
    __host__ __device__
    double operator()(const int& x, const int& y) {
           return a * x + y;
    }
};

int main(int argc, char* argv[]) {
     if (argc != 3) {
         cerr <<
          "***Numero incorreto de argumentos ***\n";
         return 1;
     }

     int n = atoi(argv[1]); // tamanho dos vetores
     int m = atoi(argv[2]); // constante

     //gerar numeros aleatorios
     thrust::host_vector<int> a(n);
     thrust::host_vector<int> b(n);
     thrust::host_vector<int> c(n);
     thrust::generate(a.begin(), a.end(), rand);
     thrust::generate(b.begin(), b.end(), rand);

     //transferimos para a GPU
     thrust::device_vector<int> d_a = a;
     thrust::device_vector<int> d_b = b;

     //transformacao

     thrust::transform(d_a.begin(), d_a.end(),
                       d_b.begin(), d_b.end(),
                       saxpy(m));

     thrust::copy(d_b.begin(), d_b.end(),
     c.begin()); 

     for (int i = 0; i < n; i++ )
         cout << setw(6) << c[i] << " = " 
          << setw(2) << m
          << "*" << setw(5) << a[i]
          << "+" << setw(5) << b[i]
          << endl;

}

Writing aula15.cu


In [2]:
!nvcc aula15.cu -o aula15

In [4]:
!./aula15 100 50

1036140795 = 50*1804289383+1036140795
463480570 = 50*846930886+463480570
2040651434 = 50*1681692777+2040651434
1975960378 = 50*1714636915+1975960378
317097467 = 50*1957747793+317097467
1892066601 = 50*424238335+1892066601
1376710097 = 50*719885386+1376710097
927612902 = 50*1649760492+927612902
1330573317 = 50*596516649+1330573317
603570492 = 50*1189641421+603570492
1687926652 = 50*1025202362+1687926652
660260756 = 50*1350490027+660260756
959997301 = 50*783368690+959997301
485560280 = 50*1102520059+485560280
402724286 = 50*2044897763+402724286
593209441 = 50*1967513926+593209441
1194953865 = 50*1365180540+1194953865
894429689 = 50*1540383426+894429689
364228444 = 50*304089172+364228444
1947346619 = 50*1303455736+1947346619
221558440 = 50*35005211+221558440
270744729 = 50*521595368+270744729
1063958031 = 50*294702567+1063958031
1633108117 = 50*1726956429+1633108117
2114738097 = 50*336465782+2114738097
2007905771 = 50*861021530+2007905771
1469834481 = 50*278722862+1469834481
822890675 = 5